In [31]:
from bs4 import BeautifulSoup
from pandas import DataFrame
from time import sleep
import urllib.request
import pandas as pd
import numpy as np
import requests
import datetime
import requests
import glob   
import os
import io
import re

In [32]:
make_model_keys=pd.read_html('https://www.kbb.com/car-make-model-list/new/view-all/')

In [33]:
df_make_model=pd.DataFrame(make_model_keys[0])

In [34]:
df_make_model=df_make_model[['Make','Model']]

In [35]:
df_make_model['Make'].nunique()

44

In [50]:
model_list=df_make_model['Model'].tolist()

In [36]:
make_list=['Toyota','Honda','Chevrolet','Ford','Mercedes','Jeep','BMW',
           'Porsche','Subaru','Nissan','Cadillac','Volkswagen','Lexus',
           'Audi','Ferrari','Volvo','Jaguar','GMC','Buick','Acura','Bentley',
           'Dodge','Hyundai','Lincoln','Mazda','Land Rover','Tesla','Ram','Kia',
           'Chrysler','Pontiac','Infiniti','Mitsubishi','Oldsmobile','Maserati','Fiat','Mini',
           'Alfa Romeo','Saab','Genesis','Suzuki','MG','Hummer']

In [38]:
collection_df = pd.DataFrame()
for f in glob.glob(r"C:\Users\Silmi\Desktop\UsedCar-PriceEstimator\data\*.csv"):
    appenddata = pd.read_csv(f)
    collection_df = collection_df.append(appenddata)

In [41]:
collection_df.shape

(21176, 16)

In [42]:
df=collection_df[collection_df['make:'].isin(make_list)]

In [43]:
df.shape

(12524, 16)

In [55]:
df_filter=df[df['model:'].isin(model_list) ]

In [56]:
df_filter.shape

(2146, 16)

In [63]:
df=df_filter.iloc[:,2:16]

In [68]:
df.tail()

,make:,model:,trim:,VIN:,condition:,odometer:,cylinders:,color:,transmission:,type:,status:,drive:,fuel:,price:
2733,Honda,Accord,LX,NaN,NaN,68940,4,NaN,automatic,NaN,clean,NaN,gas,11500.0
2746,Jeep,Wrangler,NaN,1J4FA24189L786142,excellent,108000,6,white,manual,SUV,clean,4wd,gas,18900.0
2756,Honda,Accord,LX,1HGCM66366A028797,good,108188,6,silver,automatic,sedan,clean,NaN,gas,5500.0
2764,Porsche,911,NaN,NaN,good,75000,6,red,manual,convertible,clean,rwd,gas,49000.0
2767,Honda,Civic,NaN,NaN,excellent,103000,4,brown,automatic,sedan,salvage,fwd,gas,3850.0
